In [1]:
import captum
import torch
import utils
import model.net as net
from model.data_loader import DataLoader
from torch import device as dev
from captum.attr import LayerIntegratedGradients, LayerGradientShap, LayerFeatureAblation, TokenReferenceBase, visualization
from captum_viz import interpret_sequence

In [ ]:
%load_ext autoreload

Parameter Setup

In [2]:
MODEL_DIR = 'experiments/base_model/'
DATA_DIR = 'data/'
sentences_file = DATA_DIR + 'train/sentences.txt'
labels_file = DATA_DIR + 'train/labels.txt'
weights = MODEL_DIR + 'best.pth'
params = utils.Params(MODEL_DIR+'params.json')
params.vocab_size = 25
params.number_of_classes = 10
params.cuda = torch.cuda.is_available()

Data setup

In [3]:
loader = DataLoader(DATA_DIR, params)
data = loader.load_data(['train', 'val'], DATA_DIR)
train_data = data['train']
train_data_iterator = loader.data_iterator(train_data, params, shuffle=False)
train_batch, label_batch = next(train_data_iterator)

classes = ['Extracellular', 'Plastid', 'Cytoplasm', 'Mitochondrion', 
'Nucleus', 'ER', 'Golgi', 'Membrane', 'Lysosome', 'Peroxisome']

Model setup

In [4]:
model = net.Net(params).cuda() if params.cuda else net.Net(params)
checkpoint = torch.load(weights, map_location=dev('cpu'))
model.load_state_dict(checkpoint['state_dict'])
model.eval()

Net(
  (embedding): Embedding(25, 20)
  (lstm): LSTM(20, 40, num_layers=2, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (attention): Attention(
    (linear_in): Linear(in_features=512, out_features=512, bias=False)
    (linear_out): Linear(in_features=1024, out_features=512, bias=False)
    (softmax): Softmax(dim=-1)
    (tanh): Tanh()
  )
  (fc): Linear(in_features=80, out_features=10, bias=True)
)

In [5]:
bool(model.use_attention)

False

**Visualizing Integrated Gradient**

In [6]:
sentences = open(sentences_file).read().splitlines()
labels = open(labels_file).read().splitlines()
token_reference = TokenReferenceBase(reference_token_idx=loader.pad_ind)
layer_ig = LayerIntegratedGradients(model, model.embedding)
ig_vis_data_records = []
ig_attn_vis_data_records = []

In [7]:
interpret_sequence(model, sentences[:5], train_batch, layer_ig, ig_vis_data_records)

pred:  Extracellular ( 0.76 ) , delta:  5.581566488166434e-08
pred:  Cytoplasm ( 0.64 ) , delta:  1.432005203305664e-08
pred:  Cytoplasm ( 0.69 ) , delta:  2.3265631227120664e-08
pred:  Mitochondrion ( 0.83 ) , delta:  1.6991394513610203e-08
pred:  Cytoplasm ( 0.69 ) , delta:  1.871156388166817e-08


In [8]:
visualization.visualize_text(ig_vis_data_records)

In [9]:
params.attn = 1
attn_model = net.Net(params).cuda() if params.cuda else net.Net(params)
checkpoint = torch.load(weights, map_location=dev('cpu'))
attn_model.load_state_dict(checkpoint['state_dict'])
attn_model.eval()

Net(
  (embedding): Embedding(25, 20)
  (lstm): LSTM(20, 40, num_layers=2, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (attention): Attention(
    (linear_in): Linear(in_features=512, out_features=512, bias=False)
    (linear_out): Linear(in_features=1024, out_features=512, bias=False)
    (softmax): Softmax(dim=-1)
    (tanh): Tanh()
  )
  (fc): Linear(in_features=80, out_features=10, bias=True)
)

In [10]:
bool(attn_model.use_attention)

True

In [11]:
layer_ig_attn = LayerIntegratedGradients(attn_model, attn_model.embedding)
interpret_sequence(attn_model, sentences[:5], train_batch, layer_ig_attn, ig_attn_vis_data_records)

pred:  Extracellular ( 0.99 ) , delta:  8.579375077033546e-08
pred:  Plastid ( 1.00 ) , delta:  8.487881277829956e-07
pred:  Cytoplasm ( 0.88 ) , delta:  3.6450237450580403e-07
pred:  Mitochondrion ( 1.00 ) , delta:  2.3572954965800363e-07
pred:  Cytoplasm ( 0.82 ) , delta:  7.547191882562032e-08


In [12]:
visualization.visualize_text(ig_attn_vis_data_records)

**GradientSHAP Visualization**

In [49]:
%autoreload 2

In [50]:
layer_shap = LayerGradientShap(model, model.embedding)
layer_shap_attn = LayerGradientShap(attn_model, attn_model.embedding)
shap_vis_data_records = []
shap_attn_vis_data_records = []

In [51]:
interpret_sequence(model, sentences[:5], train_batch.long(), layer_shap, shap_vis_data_records)

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)

In [ ]:
visualization.visualize_text(shap_vis_data_records)

In [ ]:
interpret_sequence(attn_model, sentences[:5], train_batch, layer_shap_attn, shap_attn_vis_data_records)

In [ ]:
visualization.visualize_text(shap_vis_attn_data_records)

**Feature Ablation Visualization**

In [41]:
model

Net(
  (embedding): Embedding(25, 20)
  (lstm): LSTM(20, 40, num_layers=2, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (attention): Attention(
    (linear_in): Linear(in_features=512, out_features=512, bias=False)
    (linear_out): Linear(in_features=1024, out_features=512, bias=False)
    (softmax): Softmax(dim=-1)
    (tanh): Tanh()
  )
  (fc): Linear(in_features=80, out_features=10, bias=True)
)

In [52]:
layer_fa = LayerFeatureAblation(model, model.embedding)
fa_vis_data_records = []

In [55]:
interpret_sequence(model, sentences[:5], model.embedding(train_batch).long(), layer_fa, fa_vis_data_records)

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)

In [ ]:
layer_fa_attn = LayerFeatureAblation(model, attn_model.embedding)
fa_attn_vis_data_records = []

In [36]:
interpret_sequence(attn_model, sentences[:5], train_batch, layer_fa_attn, fa_attn_vis_data_records)

RuntimeError: The size of tensor a (673) must match the size of tensor b (20) at non-singleton dimension 3